In [1]:
import os
import shutil
import time
from tqdm import tqdm
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd

In [3]:
# Chemin des dossiers source et destination
source_dir = "/Users/mohameddiallo/Analyse-de-donnees-et-RL-inverse-pour-la-d-tection-de-biais/ProjetEmmanuelleClaeys/Academic_Dataset_by_Generated_Photos/generated.photos"
new_images_dir = "/Users/mohameddiallo/Analyse-de-donnees-et-RL-inverse-pour-la-d-tection-de-biais/ProjetEmmanuelleClaeys/classification_verif"

# Charger le modèle PyTorch pré-entraîné
model = torch.load("/Users/mohameddiallo/Analyse-de-donnees-et-RL-inverse-pour-la-d-tection-de-biais/ProjetEmmanuelleClaeys/code/best_model_resnet101.pth")
model.eval()

# Transformation des images pour l'inférence
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Fonction pour classifier les nouvelles images
def classify_new_images(source_dir, new_images_dir, model):
    since = time.time()
    # Liste des fichiers dans le dossier source
    source_files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    # Création du dossier pour les nouvelles images si nécessaire
    if not os.path.exists(new_images_dir):
        os.makedirs(new_images_dir)
    
    # Parcours des fichiers dans le dossier source
    for file in tqdm(source_files, desc="Prédiction de nouvelles images", unit="image"):
        # Charger l'image et effectuer la prédiction
        image = Image.open(os.path.join(source_dir, file))
        image_tensor = transform(image).unsqueeze(0)
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        
        # Créer un sous-dossier pour la classe prédite si nécessaire
        predicted_class_dir = os.path.join(new_images_dir, str(predicted.item()))
        if not os.path.exists(predicted_class_dir):
            os.makedirs(predicted_class_dir)
        
        # Déplacer l'image dans le sous-dossier de la classe prédite
        shutil.move(os.path.join(source_dir, file), os.path.join(predicted_class_dir, file))
    
    time_elapsed = time.time() - since
    print(f'Classification complète en {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

# Appel de la fonction
classify_new_images(source_dir, new_images_dir, model)


Prédiction de nouvelles images: 100%|██| 10000/10000 [30:24<00:00,  5.48image/s]


Classification complète en 30m 24s


In [5]:
# Charger le modèle PyTorch pré-entraîné pour l'identification du genre
genre_model = torch.load("best_model_Gender_class.pth")
genre_model.eval()

# Transformation des images pour l'inférence
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dossier contenant les images classées par couleur de peau
base_dir = "/Users/mohameddiallo/Analyse-de-donnees-et-RL-inverse-pour-la-d-tection-de-biais/ProjetEmmanuelleClaeys/classification_verif"

# Liste des dossiers (couleurs de peau)
skin_color_folders = [folder for folder in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, folder))]

# Liste pour stocker les informations à sauvegarder dans le CSV
data = []

# Parcours des dossiers de couleur de peau
for skin_color_folder in tqdm(skin_color_folders, desc="Parcours des dossiers de couleur de peau", unit="image"): 
    # Chemin complet du dossier de couleur de peau
    skin_color_path = os.path.join(base_dir, skin_color_folder)
    
    # Liste des fichiers (images) dans le dossier
    image_files = [f for f in os.listdir(skin_color_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    
    # Parcours des images dans le dossier
    for image_file in image_files:
        # Charger l'image
        image = Image.open(os.path.join(skin_color_path, image_file))
        # Prétraitement de l'image
        image_tensor = transform(image).unsqueeze(0)
        # Prédiction du genre
        output = genre_model(image_tensor)
        _, predicted = torch.max(output, 1)
        
        # Ajouter les informations à la liste de données
        data.append({'image': image_file, 'skin_color': skin_color_folder, 'genre': predicted.item()})

# Convertir la liste de données en DataFrame pandas
df = pd.DataFrame(data)

# Enregistrer le DataFrame dans un fichier CSV
df.to_csv("informations_images.csv", index=False)

print("Les informations ont été sauvegardées dans le fichier CSV.")


Parcours des dossiers de couleur de peau: 100%|█| 5/5 [49:56<00:00, 599.25s/imag

Les informations ont été sauvegardées dans le fichier CSV.
